In [ ]:
import os
from argparse import ArgumentParser

import datasets
import torch
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
from auto_gptq.eval_tasks import TextSummarizationTask
from transformers import AutoTokenizer, GenerationConfig

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
base_model_dir = "/home/grev/models/decapoda-research_llama-13b-quant"
use_fast_tokenizer = False

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_dir, use_fast=use_fast_tokenizer)
if not tokenizer.pad_token_id:
    tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoGPTQForCausalLM.from_quantized(
    base_model_dir,
    use_triton=True,
    warmup_triton=False,
    trainable=True,
    inject_fused_attention=True,
    inject_fused_mlp=False
)
model.warmup_triton()
device = model.device


In [ ]:
from auto_gptq import AutoGPTQForCausalLM, get_gptq_peft_model
from auto_gptq.utils.data_utils import make_data_block, collate_data
from auto_gptq.utils.peft_utils import GPTQLoraConfig
from peft import TaskType


In [ ]:
model = get_gptq_peft_model(model, auto_find_all_linears=True, train_mode=False, model_id="/home/grev/code/gqtl/gptqlora/outputoscar1/checkpoint-100/adapter_model",adapter_name="oscar1")
model.print_trainable_parameters()

In [ ]:
model

In [ ]:
model.load_adapter("/home/grev/code/gqtl/gptqlora/outputoscar2/checkpoint-100/adapter_model", adapter_name="oscar2")

In [ ]:
model.set_adapter("oscar1")

In [ ]:
model.set_adapter("oscar2")

In [ ]:
model.disable_adapter()

In [ ]:
model.active_peft_config

In [ ]:
model.to("cuda")

In [ ]:
import torch

device = "cuda"


def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Input:
{input}
### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
### Instruction:
{instruction}
### Response:"""


def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=256,
    **kwargs,
):
    torch.manual_seed(42)
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.split("### Response:")[1].strip()

In [ ]:
def generate_prompt(instruction, input=None):
    return instruction

def evaluate(
    instruction,
    input=None,
    temperature=0.1,
    top_p=0.75,
    top_k=40,
    num_beams=4,
    max_new_tokens=256,
    **kwargs,
):
    torch.manual_seed(42)
    prompt = generate_prompt(instruction, input)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        num_beams=num_beams,
        no_repeat_ngram_size=3,
        **kwargs,
    )

    with torch.no_grad():
        generation_output = model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=max_new_tokens,
        )
    s = generation_output.sequences[0]
    output = tokenizer.decode(s)
    return output.strip()

In [ ]:
instruction = "Complete the below:\nThe winner of the 2023 Oscar ACTOR IN A LEADING ROLE was"

print(evaluate(instruction))

In [ ]:
instruction = "The winner of the 2023 Oscar CINEMATOGRAPHY was"

print(evaluate(instruction))

In [ ]:
with model.disable_adapter():
    instruction = "Complete the below:\nThe winner of the 2023 Oscar ACTOR IN A LEADING ROLE was"

    print(evaluate(instruction))